### Introduction

This is a notebook where I performed exploratory analysis and narrowed key insights to include in my Tableau dashboard.

The dataset used can be found [here](https://www.kaggle.com/datasets/nikdavis/steam-store-games). It is a dataset of all video games list on Steam as of May 2019. The information complete and fit for analysis (important step because not all Kaggle datasets are).

### Set up 

Loading the necessary packages

In [ ]:
!pip install google-auth google-auth-oauthlib google-auth-httplib2

In [ ]:
import pandas as pd
from google.cloud import bigquery
from google.colab import auth
from google.colab import data_table
from google.colab import drive
import os

Providing authentication to access data stored in Big Query

In [ ]:
project = 'steam-data-analysis-382023'
location = 'US'
client = bigquery.Client(project=project, location=location)
data_table.enable_dataframe_formatter()
auth.authenticate_user()

### Example query and execution

This is sample code that can be used to write then run queries.

In [ ]:
query = '''
        SELECT genre, COUNT(*) AS gamecount, AVG(price) AS priceavg
        FROM `steam-data-analysis-382023.steam.videogames`, UNNEST(SPLIT(genres, ';')) AS genre
        GROUP BY genre
        ORDER BY gamecount DESC
        '''

df = client.query(query).to_dataframe()

In [ ]:
df

### Queries for exploratory data analysis

In [ ]:
# number of developers 
'''
SELECT COUNT(DISTINCT developer)
FROM `steam-data-analysis-382023.steam.videogames`, UNNEST(SPLIT(genres, ';')) AS genre
'''

# number of games and average price by genre
'''
SELECT genre, COUNT(*) AS gamecount, AVG(price) AS priceavg
FROM `steam-data-analysis-382023.steam.videogames`, UNNEST(SPLIT(genres, ';')) AS genre
GROUP BY genre
ORDER BY gamecount DESC
'''

# games by owners group
'''
SELECT owners, count(*) AS gamecount
FROM `steam-data-analysis-382023.steam.videogames`
GROUP BY owners
'''

# number of games by release year
'''
SELECT EXTRACT(YEAR FROM release_date) AS year, COUNT(*) AS num_games
FROM `steam-data-analysis-382023.steam.videogames`
GROUP BY year
ORDER BY year DESC
'''

# number games and postitive ratings by developers 
'''
SELECT developer, COUNT(*) AS gamecount, SUM(positive_ratings) AS positives
FROM `steam-data-analysis-382023.steam.videogames`
GROUP BY developer
ORDER BY gamecount DESC
'''

# games ordered by percentage of positive ratings
'''
SELECT name, positive_ratings/(positive_ratings+negative_ratings) AS ratio
FROM `steam-data-analysis-382023.steam.videogames`
ORDER BY ratio DESC
LIMIT 10
'''

# almost 20% of games are multiplayer
'''
SELECT LOWER(categories) LIKE '%multi-player%' AS multiplayer, COUNT(*) AS gamecount
FROM `steam-data-analysis-382023.steam.videogames`
GROUP BY multiplayer
'''

# average playtimes and prices by genre
'''
SELECT genre, COUNT(*) AS gamecount, AVG(average_playtime) AS playtimemed, AVG(price) AS priceavg
FROM `steam-data-analysis-382023.steam.videogames`, UNNEST(SPLIT(genres, ';')) AS genre
GROUP BY genre
ORDER BY playtimemed DESC
'''

"\nSELECT *\nFROM `steam-data-analysis-382023.steam.videogames` v \nLEFT JOIN (\n  SELECT steam_appid, REGEXP_REPLACE(about_the_game, '<[^>]+>', '') AS cleaned_description\n  FROM `steam-data-analysis-382023.steam.descriptions`\n) d \nON v.appid = d.steam_appid\n"

### Queries for key insights that I want to highlight in the dashboard

In [ ]:
# Most of the top reviewed games are under the action genre
'''
SELECT name, developer, positive_ratings, negative_ratings, positive_ratings+negative_ratings AS ratings, LOWER(genres) LIKE '%action%' AS action
FROM `steam-data-analysis-382023.steam.videogames`
ORDER BY ratings DESC
LIMIT 10
'''

# multiplayer games have triple the average playtimes
'''
SELECT LOWER(categories) LIKE '%multi-player%' AS multiplayer, AVG(average_playtime) AS median_playtime
FROM `steam-data-analysis-382023.steam.videogames`
GROUP BY multiplayer
'''

# less than 20 percent of games are over $10
'''
SELECT
  CASE
    WHEN price = 0 THEN 'Free'
    WHEN price > 0 AND price < 10 THEN 'Under $10'
    ELSE 'Over $10'
  END AS pricegroup,
  COUNT(*) AS gamecount
FROM `steam-data-analysis-382023.steam.videogames`
GROUP BY pricegroup
'''

# among top publishers, valve have the lowest prices but highest playtimes
'''
SELECT publisher, COUNT(*) AS gamecount, AVG(average_playtime) AS playtimeavg, AVG(price) AS priceavg
FROM `steam-data-analysis-382023.steam.videogames`, UNNEST(SPLIT(publisher, ';')) AS publisher
WHERE publisher IN ('Valve','Ubisoft','Bethesda Softworks','Square Enix','2K','CD PROJEKT RED','BANDAI NAMCO Entertainment','Rockstar Games','SEGA','Activision')
GROUP BY publisher
ORDER BY playtimeavg DESC
'''

### Code to read queries and convert results to pandas dataframe

Store queries in variables

In [ ]:
query_1 = '''
          SELECT name, developer, positive_ratings, negative_ratings, positive_ratings+negative_ratings AS ratings, LOWER(genres) LIKE '%action%' AS action
          FROM `steam-data-analysis-382023.steam.videogames`
          ORDER BY ratings DESC
          LIMIT 10
          '''

query_2 = '''
          SELECT
            CASE
              WHEN price = 0 THEN 'Free'
              WHEN price > 0 AND price < 10 THEN 'Under $10'
              ELSE 'Over $10'
            END AS pricegroup,
            COUNT(*) AS gamecount
          FROM `steam-data-analysis-382023.steam.videogames`
          GROUP BY pricegroup
          '''

query_3 = '''
          SELECT LOWER(categories) LIKE '%multi-player%' AS multiplayer, AVG(average_playtime) AS median_playtime
          FROM `steam-data-analysis-382023.steam.videogames`
          GROUP BY multiplayer
          '''

query_4 = '''
          SELECT publisher, COUNT(*) AS gamecount, AVG(average_playtime) AS playtimeavg, AVG(price) AS priceavg
          FROM `steam-data-analysis-382023.steam.videogames`, UNNEST(SPLIT(publisher, ';')) AS publisher
          WHERE publisher IN ('Valve','Ubisoft','Bethesda Softworks','Square Enix','2K','CD PROJEKT RED','BANDAI NAMCO Entertainment','Rockstar Games','SEGA','Activision')
          GROUP BY publisher
          ORDER BY playtimeavg DESC
          '''

query_5 = '''
          SELECT *
          FROM `steam-data-analysis-382023.steam.videogames` v 
          LEFT JOIN (
            SELECT steam_appid, REGEXP_REPLACE(about_the_game, '<[^>]+>', '') AS cleaned_description
            FROM `steam-data-analysis-382023.steam.descriptions`
          ) d 
          ON v.appid = d.steam_appid
          '''

Run each query and store result dataframes in variables

In [ ]:
steam_games = client.query(query_1).to_dataframe()
steam_prices = client.query(query_2).to_dataframe()
steam_multiplayer = client.query(query_3).to_dataframe()
steam_publisher = client.query(query_4).to_dataframe()
steam_all = client.query(query_5).to_dataframe()

A function to loop through each dataframe and write them to Google Drive

In [ ]:
def toDrive(df_list):
    drive.mount('/drive')
    for i, df in enumerate(df_list):
        filename = f'dataframe_{i}.csv'
        filepath = os.path.join('/drive/My Drive/DataProjects/SteamDashboard', filename)
        df.to_csv(filepath)

In [ ]:
toDrive([steam_games,steam_prices,steam_multiplayer,steam_publisher,steam_all])

Drive already mounted at /drive; to attempt to forcibly remount, call drive.mount("/drive", force_remount=True).
